In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.evapo import *

workhorse = True

if workhorse:
    origin = 'Aldhani/eoagritwin/'
else:
    origin = ''

import re
from other_repos.pyDMS.pyDMS.pyDMS import *

In [2]:
reduce_force_to_validmonths(f'/data/{origin}force/output/Guzinski/1TileTest/',3,8)
files = getFilelist(f'/data/{origin}force/output/Guzinski/1TileTest/', '.tif', deep=True) 
tiles = ['X0067_Y0042', 'X0068_Y0042']
unique_days = get_forceTSI_output_DOYS(files)

In [ ]:
# get dates from FORCE 10 day interpolations and make stacked tifs


for tile in tiles:
  for unique_day in unique_days:
    stack_tifs(sorted([file for file in files if unique_day in file and tile in file]), f'/data/{origin}force/output/Guzinski/1TileTest/{tile}_{unique_day}.tif')

In [3]:
files_2Tiles = sorted([file for file in files for tile in tiles if tile in file and unique_days[12] in file])
files_2Tiles = [files_2Tiles[i] for i in [0,10,1,11,2,12,3,13,4,14,5,15,6,16,7,17,8,18]]


In [4]:
force_order_BGRBNR(files_2Tiles)

ValueError: invalid literal for int() with base 10: '365_HL_TSA_SEN2L_BLU_TSI_20190630'

/data/Aldhani/eoagritwin/force/output/Guzinski/1TileTest/Force_X_from_67_to_68_Y_from_42_to_42/
single vrts created


In [ ]:
 # tiles = list(set([file.split('output/')[-1].split('/')[1].split('/')[0] for file in list_of_forcefiles]))
    

        # make paths in vrts relative
        vrts = getFilelist(outDir, '.vrt')
        for vrt in vrts:
            convertVRTpathsTOrelative(vrt)
        nums = [int(vrt.split('_')[-1].split('.')[0]) for vrt in vrts]
        vrts_sorted = sortListwithOtherlist(nums, vrts)[-1]
        print('paths in vrts made relative')
        
        vrt = gdal.BuildVRT(f'{outDir}{force_folder_name}_Cube.vrt', vrts_sorted, separate = True)
        vrt = None
        # convertVRTpathsTOrelative(f'{outDir}{force_folder_name}_Cube.vrt')
        print('overlord vrt created')
        if pyramids:
            # build pyramids
            vrtPyramids(f'{outDir}{force_folder_name}_Cube.vrt')
            print('VRT created with pyramids')
    else:
        print('Vrt might already exist - please check!!')

In [ ]:
highResFilename1 = '/data/Aldhani/eoagritwin/force/output/Guzinski/1TileTest/X0067_Y0042_20190630.tif'
highResFilename2 = '/data/Aldhani/eoagritwin/force/output/Guzinski/1TileTest/X0068_Y0042_20190630.tif'
lowResFilename = '/data/Aldhani/eoagritwin/et/Sentinel3/tiffs/LST/daily_observations_all/2019/Daily_LST_means_2019_July.tif'

outputFilename = '/data/Aldhani/eoagritwin/et/Sentinel3/tiffs/LST/test6_1.tif'

In [ ]:
useDecisionTree = True

commonOpts = {"highResFiles":               [highResFilename1, highResFilename2],#[highResFilename],
                "lowResFiles":              [lowResFilename, lowResFilename],
                "lowResQualityFiles":         [],# [lowResMaskFilename],
                "lowResGoodQualityFlags":     [],#[255],
                "cvHomogeneityThreshold":     0,
                "movingWindowSize":           15,
                "disaggregatingTemperature":  True}
dtOpts =     {"perLeafLinearRegression":    True,
                "linearRegressionExtrapolationRatio": 0.25}
sknnOpts =   {'hidden_layer_sizes':         (10,),
                'activation':                 'tanh'}
nnOpts =     {"regressionType":             REG_sklearn_ann,
                "regressorOpt":               sknnOpts}

start_time = time.time()

if useDecisionTree:
    opts = commonOpts.copy()
    opts.update(dtOpts)
    disaggregator = DecisionTreeSharpener(**opts)
else:
    opts = commonOpts.copy()
    opts.update(nnOpts)
    disaggregator = NeuralNetworkSharpener(**opts)

In [ ]:
print("Training regressor...")
disaggregator.trainSharpener()
print("Sharpening...")
downscaledFile = disaggregator.applySharpener(highResFilename1, lowResFilename)
print("Residual analysis...")
residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                               # lowResMaskFilename,
                                                                doCorrection=True)
print("Saving output...")
highResFile = gdal.Open(highResFilename1)
if correctedImage is not None:
    outImage = correctedImage
else:
    outImage = downscaledFile
# outData = utils.binomialSmoother(outData)
outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                        outImage.GetGeoTransform(),
                        outImage.GetProjection(),
                        outputFilename)
residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                            residualImage.GetGeoTransform(),
                            residualImage.GetProjection(),
                            os.path.splitext(outputFilename)[0] + "_residual" +
                            os.path.splitext(outputFilename)[1])

outFile = None
residualFile = None
downsaceldFile = None
highResFile = None

print(time.time() - start_time, "seconds")